In [30]:
# import necessary libraries

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib as plt
url = 'https://github.com/knnthfrmn/dsc25_group.git'

In [41]:
# load all data

unemployment_data = pd.read_csv('data/global_unemployment_data.csv')
happiness_2015 = pd.read_csv('data/happiness data/2015.csv')
happiness_2016 = pd.read_csv('data/happiness data/2016.csv')
happiness_2017 = pd.read_csv('data/happiness data/2017.csv')
happiness_2018 = pd.read_csv('data/happiness data/2018.csv')
happiness_2019 = pd.read_csv('data/happiness data/2019.csv')

# add year column to happiness dataframes

happiness_2015['year'] = 2015
happiness_2016['year'] = 2016
happiness_2017['year'] = 2017
happiness_2018['year'] = 2018
happiness_2019['year'] = 2019

# merge all happiness dataframes

happiness_data = pd.concat(happiness_2015,happiness_2016,happiness_2017,happiness_2018,happiness_2019)
happiness_data

TypeError: concat() takes 1 positional argument but 5 were given